# Загрузка пакетов и данных

Для начала надо загрузить необходимые пакеты и данные.

In [0]:
!pip install profanity_check  # Проверка оскорбительности

     |████████████████████████████████| 2.4MB 4.6MB/s 


In [0]:

!pip install spacy  # Для анализа текста

In [0]:
# Понадобится для всего
import pandas as pd

# Понадобится для изучения грубости высказываний
from collections import Counter
from profanity_check import predict

# Понадобится для предсказания персонажа по тексту фразы
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import TransformerMixin 
from sklearn.pipeline import Pipeline
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

In [0]:
df = pd.read_csv('/content/drive/My Drive/hw2/All-seasons.csv')

# Как выглядит датафрейм
df.head()

,Season,Episode,Character,Line
0,10,1,Stan,"You guys, you guys! Chef is going away. \n"
1,10,1,Kyle,Going away? For how long?\n
2,10,1,Stan,Forever.\n
3,10,1,Chef,I'm sorry boys.\n
4,10,1,Stan,"Chef said he's been bored, so he joining a gro..."


Итак, мы имеем датасет, в котором каждую строчку занимает одна фраза персонажа, указан персонаж, а также серия и сезон. Последние два параметра я использовать не буду. Также я буду изучать только топ-10 по количеству высказываний персонажей.

Создам список главных персонажей (топ-10)

In [0]:
main_characters = []
for i in range(10):
    main_characters.append(Counter(df['Character']).most_common(10)[i][0])

In [0]:
# Посмотрим на состав главных персонажей
main_characters

['Cartman',
 'Stan',
 'Kyle',
 'Butters',
 'Randy',
 'Mr. Garrison',
 'Chef',
 'Kenny',
 'Sharon',
 'Mr. Mackey']

In [0]:
# Сделаем датасет только с главными героями
dfmc = df[df['Character'].isin(main_characters)]
len(dfmc)

33917

Теперь поставлю себе задачи.<br/>
Для начала просто интересно посмотреть на процент оскорбительных фраз.<br/>
Задачи: 
1. Какой процент высказываний оскорбителен?
2. У какого персонажа (из главных) этот процент наибольший?

Потом основная задача - написать классификатор, который по фразе<br/> определит героя.
<br/>
Задачи: 
3. Обучить модель на тренировочной выборке
4. Проверить её с помощью тестовой выборки
5. Проверить на известных коронных фразах.

# Оскобительность высказываний

In [0]:
# предсказываем для каждой фразы, оскорбительна она или нет
predictions = []
for row in df.Line:
    predictions.append(int(predict([row])))  # функция из profanity_check

In [0]:
df['Swear'] = predictions

In [0]:
a = len(df[df['Swear'] == 1])
b = len(df[df['Swear'] == 0])
print(f'{round(a/(a+b), 3)*100}% высказываний оскорбительны')

7.1% высказываний оскорбительны


Ответ на задачу 1: 7.1% высказываний оскорбительны.

In [0]:
# Теперь посмотрю этот процент по персонажам
hero_profanity = {}
for hero in main_characters:
    a = len(df[df['Swear'] == 1 ][ df['Character'] == hero])  # Оскорбительные
    b = len(df[df['Swear'] == 0 ][ df['Character'] == hero])  # Допустимые

    print(f'{round(a*100/(a+b), 2)}% высказываний {hero} оскорбительны')

    hero_profanity[round(a*100/(a+b), 2)] = hero

12.87% высказываний Cartman оскорбительны
6.59% высказываний Stan оскорбительны
7.59% высказываний Kyle оскорбительны
4.65% высказываний Butters оскорбительны
6.0% высказываний Randy оскорбительны
9.58% высказываний Mr. Garrison оскорбительны
7.2% высказываний Chef оскорбительны
13.73% высказываний Kenny оскорбительны
3.02% высказываний Sharon оскорбительны
5.53% высказываний Mr. Mackey оскорбительны


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


In [0]:
max_profanity = max(hero_profanity.keys())
swindler = hero_profanity[max_profanity]
print(f"Главный грубиян = это {swindler}. {max_profanity}% его фраз грубые.")

Главный грубиян = это Kenny. 13.73% его фраз грубые.


Вот и ответ на задачу 2. Достаточно интересный результат.

# Предсказание персонажа

Теперь займёмся моделью, которая будет предсказывать персонажа по фразе.

In [0]:
# Список знаков препинания
punctuations = string.punctuation

# Список стоп-слов (то есть слов, которые удалим из данных, они засоряют их)
# Эти слова встречаются часто и несут мало информации для классификации
nlp = spacy.load('en')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Загрузим из spaCy модель для предварительного анализа английского языка,
# то бишь аннотации 
parser = English()

class predictors(TransformerMixin):
    """
    Хотим сделать красиво, с пайплайном, так что вот класс "предикторы",
    который будет трансформировать текст (предиктор при обучении), делая все
    буквы строчными и убирая пробелы.
    """

    def transform(self, X, **transform_params):
        # все буквы строчные, без пробелов
        return [text.strip().lower() for text in X]

    # то, что ниже, надо для пайплайна
    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

def spacy_tokenizer(sentence):
    """
    Функция для токенизирования предложения, т.е. приведения в подходящий вид.
    (без пунктуации, без "стоп-слов", от слов остаются только их корни)
    """

    # Парсим предложение с помощью spaCy
    mytokens = parser(sentence)

    # Оставляем у слов только корни (это называется лемматизация) 
    mytokens = [word.lemma_.lower().strip()
                if word.lemma_ != "-PRON-"  # Так (c -PRON-) рекомендуют делать
                else word.lower_ for word in mytokens]

    # Убираем засоряющие текст "стоп-слова"
    mytokens = [word for word in mytokens
                if word not in stop_words and word not in punctuations]

    return mytokens

In [0]:
# Строку текста - в числа
bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)

In [0]:
X = dfmc.Line
y = dfmc.Character
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [0]:
# Интересно использовать несколько моделей
LRclassifier = LogisticRegression()  # Логистическая регрессия
RFclassifier = RandomForestClassifier()  # Случайный лес
SVMclassifier = LinearSVC()  # Вариация метода опорных векторов

# Ниже инициализируем пайплайны для каждого из классификаторов
LRpipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', bow_vector),
                 ('classifier', LRclassifier)])

RFpipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', bow_vector),
                 ('classifier', RFclassifier)])

SVMpipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', bow_vector),
                 ('classifier', RFclassifier)])

In [0]:
LRpipe.fit(X_train, y_train)

In [0]:
RFpipe.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('cleaner', <__main__.predictors object at 0x7f48b58750f0>),
                ('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 t...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_im

In [0]:
SVMpipe.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('cleaner', <__main__.predictors object at 0x7f48b58750b8>),
                ('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 t...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_im

In [0]:
# Предсказываем на тестовом датасете
LRpredicted = LRpipe.predict(X_test)
RFpredicted = RFpipe.predict(X_test)
SVMpredicted = SVMpipe.predict(X_test)

# Accuracy моделей
print("Accuracy логистической регресии:",
      metrics.accuracy_score(y_test, LRpredicted))

print("Accuracy случайного леса:",
      metrics.accuracy_score(y_test, RFpredicted))

print("Accuracy классификации методом опорных векторов:",
      metrics.accuracy_score(y_test, SVMpredicted))

Accuracy логистической регресии: 0.42845911949685533
Accuracy случайного леса: 0.3902319182389937
Accuracy классификации методом опорных векторов: 0.3902319182389937


С помошью лучшей модели предскажем коронные фразы некоторых персонажей

In [0]:
LRpipe.predict(["Oh my god, they've killed Kenny!"])

array(['Stan'], dtype=object)

In [0]:
LRpipe.predict(["You bastards!"])

array(['Kyle'], dtype=object)

In [0]:
LRpipe.predict(["I'm not fat, I'm big boned!"])

array(['Cartman'], dtype=object)

In [0]:
LRpipe.predict(["Hello there, children"])

array(['Chef'], dtype=object)

In [0]:
LRpipe.predict(["Oh, hamburgers!"])

array(['Butters'], dtype=object)